In [54]:
import requests
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import os

In [55]:
estados = ['aguascalientes', 'baja-california-norte','baja-california-sur','campeche','chiapas','chihuahua','cdmx','coahuila-de-zaragoza',
           'colima','durango','estado-de-mexico','guanajuato','guerrero', 'hidalgo','jalisco','michoacan','morelos','nayarit',
           'nuevo-leon','oaxaca', 'puebla','queretaro','quintana-roo','san-luis-potosi-slp','sinaloa','sonora','tabasco','tamaulipas',
           'tlaxcala','veracruz','yucatan','zacatecas']

estados_expresion = ['Aguascalientes', 'Baja California Norte','Baja California Sur','Campeche','Chiapas','Chihuahua','Ciudad de México - CDMX',
                    'Coahuila','Colima','Durango','Estado de México - EDOMEX','Guanajuato','Guerrero','Hidalgo','Jalisco',
                    'Michoacán','Morelos','Nayarit','Nuevo León','Oaxaca','Puebla','Querétaro','Quintana Roo','San Luis Potosí - SLP',
                    'Sinaloa','Sonora','Tabasco','Tamaulipas','Tlaxcala','Veracruz','Yucatán','Zacatecas']

data = []

In [56]:
def obtener_contenido_pagina(url):
    pagina = requests.get(url)
    return pagina.content

In [57]:
def crear_soup(html):
    return BeautifulSoup(html, "html.parser")

In [58]:
def procesar_pagina(soup, estado, fecha):

    litros = []
    kilos = []
    tanques20kg = []
    
    patron_estados =  "|".join(re.escape(estado) for estado in estados_expresion)
    patron_precio = r"\$(\d+\.\d+)"

    litro =  r"El precio por litro de gas en ({}) es de {}".format(patron_estados, patron_precio)
    concuerdalitro = re.search(litro, soup.get_text())
    preciolitro = concuerdalitro.group(2)
    litros.append(float(preciolitro))


    kilo = r"El precio por kilo de gas en ({}) es de {}".format(patron_estados, patron_precio)
    concuerdakilo = re.search(kilo, soup.get_text())
    preciokilo = concuerdakilo.group(2)
    kilos.append(float(preciokilo))

    tanque20kg = r"El tanque de 20kg vale \$(\d+\.\d+)"
    concuerdatanque20gk = re.search(tanque20kg, soup.get_text())
    preciotanque20kg = concuerdatanque20gk.group(1)
    tanques20kg.append(float(preciotanque20kg))

    #fecha = datetime.datetime.now().strftime("%d/%m/%Y")
    
    data.append({
            "Estado": estado,
            "Fecha": fecha,
            "precio_litros": litros[0],
            "precio_kilos": kilos[0],
            "precio_tanque20kg": tanques20kg[0]
        }) 

In [64]:
def manejar_paginacion(url_base):
   fecha = datetime.datetime.now().strftime("%d-%m-%Y")
   for estado in estados:
      url = url_base + estado
      contenido_pagina = obtener_contenido_pagina(url)
      soup = crear_soup(contenido_pagina)
      procesar_pagina(soup, estado, fecha)
   
   dataframe = pd.DataFrame(data)
   dataframe.to_csv("../csv" + "/base.csv", index=False)

In [63]:
url_base = "https://www.gasolinamx.com/gas/precio-gas-"
manejar_paginacion(url_base)